In [ ]:
] activate .

## Version 3.10.2 (works)

In [ ]:
] add ModelingToolkit#v3.10.2

In [ ]:
using ModelingToolkit

In [ ]:
var = @variables x y

In [4]:
    using ModelingToolkit: get_variables,
                            gradient,
                            simplify,
                            to_symbolic,
                            Operation

function _halfspace(expr::Operation, vars=get_variables(expr); N::Type{<:Real}=Float64)

    # find sense and normalize
    if expr.op == <
        a, b = expr.args
        sexpr = simplify(a - b)

    elseif expr.op == >
        a, b = expr.args
        sexpr = simplify(b - a)

    elseif (expr.op == |) && (expr.args[1].op == <)
        a, b = expr.args[1].args
        sexpr = simplify(a - b)

    elseif (expr.op == |) && (expr.args[2].op == <)
        a, b = expr.args[2].args
        sexpr = simplify(a - b)

    elseif (expr.op == |) && (expr.args[1].op == >)
        a, b = expr.args[1].args
        sexpr = simplify(b - a)

    elseif (expr.op == |) && (expr.args[2].op == >)
        a, b = expr.args[2].args
        sexpr = simplify(b - a)

    else
        throw(ArgumentError("expected an expression describing a half-space, got $expr"))
    end

    # compute the linear coefficients by taking first order derivatives
    coeffs = [N(α.value) for α in gradient(sexpr, collect(vars))]

    # get the constant term by expression substitution
    dvars = Dict(to_symbolic(vi) => zero(N) for vi in vars)
    β = -N(ModelingToolkit.SymbolicUtils.substitute(to_symbolic(sexpr), dvars, fold=true))

    return coeffs, β
end

_halfspace (generic function with 2 methods)

In [ ]:
expr = x + 2y <= 3x + 4
_halfspace(expr, var)

In [ ]:
var = @variables x y
_halfspace(x <= 1.0, var)

# v3.13.0 (doesn't work)

In [1]:
] add ModelingToolkit#v3.13.0

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
No Changes to `~/Projects/escritoire/2020/Week30/Project.toml`
No Changes to `~/Projects/escritoire/2020/Week30/Manifest.toml`


In [2]:
using ModelingToolkit

┌ Info: Precompiling ModelingToolkit [961ee093-0014-501f-94e3-6117800e7a78]
└ @ Base loading.jl:1278


In [5]:
var = @variables x y
_halfspace(x <= 1.0, var)

ErrorException: type Int64 has no field value

# v3.14.0 (works again)

In [ ]:
] add ModelingToolkit#v3.14.0

In [ ]:
using ModelingToolkit

In [ ]:
var = @variables x y
_halfspace(x <= 1.0, var)